# Data Export for (https://github.com/albblgb/Deep-Steganalysis.git)

In [1]:
import os
import sys

import shutil

import numpy as np
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets
from model_xray.config_classes import *
from PIL import Image

x_train = 23

trainset_names = [f'mobilenets_train_xs0{x_train}_imsize256']

testset_names = [
    'famous_le_100m_test_benigns_imsize256',
] + [
    f'famous_le_100m_test_xs{x_train}_imsize256'
]

requested_datasets = trainset_names + testset_names

datasets = retreive_datasets(requested_datasets)

def output_imgs(img_arrs, path, output_as_rgb=False):
    os.makedirs(path, exist_ok=True)
    for i, img_arr in enumerate(img_arrs):
        img = Image.fromarray(img_arr)
        if output_as_rgb:
            img = img.convert('RGB')
        img.save(f'{path}/{i+1}.png')

def output_dataset(X,y, benign_path, mal_path, reset_dirs=False):
    if reset_dirs:
        if os.path.exists(benign_path):
            shutil.rmtree(benign_path)
        if os.path.exists(mal_path):
            shutil.rmtree(mal_path)

    os.makedirs(benign_path, exist_ok=True)
    os.makedirs(mal_path, exist_ok=True)

    X_benign = X[y == PreprocessedImageDatasetLabel.BENIGN]
    output_imgs(X_benign, benign_path)
    X_mal = X[y == PreprocessedImageDatasetLabel.ATTACKED]
    output_imgs(X_mal, mal_path)

def merge_datasets(datasets):
    Xs = []
    ys = []
    for X,y in datasets:
        Xs.append(X)
        ys.append(y)
    return np.vstack(Xs), np.hstack(ys)

train_datasets = [datasets[name] for name in trainset_names]
test_datasets = [datasets[name] for name in testset_names]
X_train, y_train = merge_datasets(train_datasets)
X_test, y_test = merge_datasets(test_datasets)

def fill_deep_steganalysis_data_dirs(*, X_train, y_train, X_test, y_test, root_dir='', X_val=None, y_val=None):
    train_path = f'{root_dir}/train_data_dir'
    test_path = f'{root_dir}/test_data_dir'
    val_path = f'{root_dir}/val_data_dir'

    output_dataset(X_train, y_train, f'{train_path}/cover', f'{train_path}/stego', reset_dirs=True)
    output_dataset(X_test, y_test, f'{test_path}/cover', f'{test_path}/stego', reset_dirs=True)
    if X_val is None or y_val is None:
        X_val, y_val = X_test, y_test
    
    output_dataset(X_val, y_val, f'{val_path}/cover', f'{val_path}/stego', reset_dirs=True)

# root_dir = '/mnt/exdisk2/image_steganalysis/deep_steganalysis'
fill_deep_steganalysis_data_dirs(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, root_dir='/mnt/exdisk2/image_steganalysis/deep_steganalysis')

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.
